In [27]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [28]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [29]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [30]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [31]:
from bs4 import BeautifulSoup
import requests
import re

### 도구 정의

In [32]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=3)

In [33]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""

    result = ""
    
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [34]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

### Agent 프롬프트 작성

In [35]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [36]:
tools = [search_keyword, python_repl_tool, naver_news_crawl]

### Agent 생성

In [37]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [38]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=False,                   # 중간 단계 출력
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

stream() 메소드는 에이전트의 중간 단계를 스트리밍 출력합니다. <br>

stream() 출력은 (Action, Obervation) 쌍 사이에서 번갈아가면서 나타납니다. <br>
최종적으로 에이전트가 목표를 달성했으면 답변으로 마무리됩니다. <br>

1. Action 출력
2. Observation 출력
3. Action 출력
4. Observation 출력

... 목표 달성 까지 계속 ... <br>

최종 목표가 달성되면 에이전트는 최종 답변을 출력합니다. <br>

| 단계 | 내용 |
|------|-----------|
| Action | actions : AgentAction 객체가 호출되면서 어떤 도구를 사용할지 결정을 해준다. LLM이 실행하고 실행 결과를  Obervation의 messages에 넣는다. <br> messages :  action 호출에 해당하는 채팅 메시지 |
| Obervation | steps : 현재 액션과 그 관찰을 포함한 에이전트가 지금까지 수행한 작업의 기록. <br> messages : 도구 호출의 실행 결과. 이 결과를 보고 최종 답변으로 만들게 된다. |
| Final Answer | output : Agent Finish. <br> messages : 최종 답변. |

(1) action 단계에서는 AgentAction 객체가 호출되면서 LLM이 어떤 도구를 사용할지 결정을 해준다.
    LLM이 도구를 결정을 해주면 실행해서 Observation에 실행결과가 담긴다. messagees부분에 담긴다. <br>
(2) Observation에 단계에서는 messagees부분에 담긴 결과를 보고 최종 답변으로 만든다. <br>
(3) Final Answer : 

In [39]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성해 주세요'})

for step in result:
    print(step)
    print('==' * 50)

{'actions': [ToolAgentAction(tool='python_repl_tool', tool_input={'code': 'print(10 + 20)'}, log="\nInvoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_vADxJ1dRG0qwuTvBglLml9rf', 'function': {'arguments': '{"code":"print(10 + 20)"}', 'name': 'python_repl_tool'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac'}, id='run-0d026d41-259d-4612-a83d-50eb240f12dc', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(10 + 20)'}, 'id': 'call_vADxJ1dRG0qwuTvBglLml9rf', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'python_repl_tool', 'args': '{"code":"print(10 + 20)"}', 'id': 'call_vADxJ1dRG0qwuTvBglLml9rf', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_vADxJ1dRG0qwuTvBglLml9rf')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_c

In [40]:
result = agent_executor.stream({'input': '대구 동성로 떡볶이 대해 검색해주세요'})

for step in result:
    print()
    print(step)
    print('==' * 50)


{'actions': [ToolAgentAction(tool='search_keyword', tool_input={'query': '대구 동성로 떡볶이'}, log="\nInvoking: `search_keyword` with `{'query': '대구 동성로 떡볶이'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ijVaoH9yccDM4xa1Ca0aQsBu', 'function': {'arguments': '{"query":"대구 동성로 떡볶이"}', 'name': 'search_keyword'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac'}, id='run-e3fe607a-6b04-4c0d-8221-94fceb45a4c5', tool_calls=[{'name': 'search_keyword', 'args': {'query': '대구 동성로 떡볶이'}, 'id': 'call_ijVaoH9yccDM4xa1Ca0aQsBu', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_keyword', 'args': '{"query":"대구 동성로 떡볶이"}', 'id': 'call_ijVaoH9yccDM4xa1Ca0aQsBu', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_ijVaoH9yccDM4xa1Ca0aQsBu')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id

In [41]:
result = agent_executor.stream({'input': '대구 동성로 떡볶이 대해 검색해주세요'})

i = 1

for step in result:
    print('==' * 30)
    # print(f'[step] ({i}) : \n', step)
    print(f"[step] ({i}): \n")

    if 'actions' in step:        
        print(f'===== [actions] 시작 =====')
        
        for action in step['actions']:
            print(f"action type: {type(action)}")
            print(action)
        print(f'===== [actions]  끝 =====')

        print(f'===== [messages] 시작 =====')

        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")

            if hasattr(message, 'tool_calls'):
                print('도구 호출 이름: ', message.tool_calls[0]['name'])
                print('도구 호출 인자: ', message.tool_calls[0]['args'])
                print('도구 호출 아이디: ', message.tool_calls[0]['id'])
                print('도구 호출 타입: ', message.tool_calls[0]['type'])
            else:
                print('도구 호출 없음')
        print(f'===== [messages] 끝 =====')
    
    elif 'steps' in step:        
        print(f'===== [steps] 시작 =====')

        for agent_step in step['steps']:
            print(f" 사용 도구 이름 agent_step.action.tool: {agent_step.action.tool}")
            print(f" 입력 쿼리 agent_step.action.tool_input: {agent_step.action.tool_input}")
            print(f" Log: {agent_step.action.log}")
            # print(f"(4) Observation: {agent_step.observation}")
            print(f" Observation: \n")

            if len(agent_step.observation)>0:
                for observation in agent_step.observation:
                    print(observation)

        print(f'===== [steps]  끝 =====')

        print(f'===== [messages] 시작 =====')
        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")

            if hasattr(message, 'tool_calls'):
                print('도구 호출 이름: ', message.tool_calls[0]['name'])
                print('도구 호출 인자: ', message.tool_calls[0]['args'])
                print('도구 호출 아이디: ', message.tool_calls[0]['id'])
                print('도구 호출 타입: ', message.tool_calls[0]['type'])
            else:
                print('도구 호출 없음')
        print(f'===== [messages] 끝 =====')

    
    elif 'output' in step:        
        print(f'===== [output] 시작 =====')
        print(f"[output]: {step['output']}")  
        print(f'===== [output]  끝 =====')

        print(f'===== [messages] 시작 =====')
        for message in step['messages']:
            print(f"메시지 타입: {type(message).__name__}")
            print(f"내용 Message: {message.content}")
        print(f'===== [messages] 끝 =====')

    i=i+1

[step] (1): 

===== [actions] 시작 =====
action type: <class 'langchain.agents.output_parsers.tools.ToolAgentAction'>
tool='search_keyword' tool_input={'query': '대구 동성로 떡볶이'} log="\nInvoking: `search_keyword` with `{'query': '대구 동성로 떡볶이'}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_KzgiTnuUjLsEHbAPrG4rLuI6', 'function': {'arguments': '{"query":"대구 동성로 떡볶이"}', 'name': 'search_keyword'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac'}, id='run-424fc9dc-713b-4daa-90cd-eac014e0648f', tool_calls=[{'name': 'search_keyword', 'args': {'query': '대구 동성로 떡볶이'}, 'id': 'call_KzgiTnuUjLsEHbAPrG4rLuI6', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_keyword', 'args': '{"query":"대구 동성로 떡볶이"}', 'id': 'call_KzgiTnuUjLsEHbAPrG4rLuI6', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_KzgiTnuUjLsEHbAPrG4rLuI6'
===== [acti

In [42]:
result = agent_executor.stream({'input': '"네이버 뉴스 기사의 내용을 크롤링해주세요: https://n.news.naver.com/mnews/article/002/0002372963"'})

In [43]:
result = agent_executor.stream({'input': '10+20의 결과는?'})

In [44]:
""" 
{

  'actions': 
  [
    ToolAgentAction
    (
      tool='search_keyword', 

      tool_input={'query': '대구 동성로 떡볶이'}, 

      log="\nInvoking: `search_keyword` with `{'query': '대구 동성로 떡볶이'}`\n\n\n", 

      message_log=
      [
        AIMessageChunk
        (
          content='', 
          additional_kwargs=
           {
              'tool_calls': [
                                 {
                                    'index': 0, 
                                    'id': 'call_ipWysUL54d6KgDfUXjgRRsYP', 
                                    'function': {'arguments': '{"query":"대구 동성로 떡볶이"}', 'name': 'search_keyword'}, 
                                    'type': 'function'
                                 }
                            ]
           }, 

           response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d'}, 
 
          id='run-c94d8c7f-29bf-4873-b108-8bf4349dfc49', 

          tool_calls=[ 
                           {
                              'name': 'search_keyword', 'args': {'query': '대구 동성로 떡볶이'}, 
                              'id': 'call_ipWysUL54d6KgDfUXjgRRsYP', 'type': 'tool_call'
                           }
                        ], 

          tool_call_chunks=[
                                     {
                                         'name': 'search_keyword', 
                                         'args': '{"query":"대구 동성로 떡볶이"}', 
                                         'id': 'call_ipWysUL54d6KgDfUXjgRRsYP', 
                                         'index': 0, 'type': 'tool_call_chunk'
                                     }
                                  ]
        )
      ],    // end message_log

    tool_call_id='call_ipWysUL54d6KgDfUXjgRRsYP'

    ) // ToolAgentAction
  ], 


  'messages': 
  [
    AIMessageChunk
    (
      content='', 
      additional_kwargs=
       {
           'tool_calls': [
                             {
                                 'index': 0, 
                                 'id': 'call_ipWysUL54d6KgDfUXjgRRsYP', 
                                 'function': {'arguments': '{"query":"대구 동성로 떡볶이"}', 'name': 'search_keyword'}, 
                                 'type': 'function'
                             }
                          ]
       }, 

      response_metadata= {
                            	'finish_reason': 'tool_calls', 
                            	'model_name': 'gpt-4o-mini-2024-07-18', 
                            	'system_fingerprint': 'fp_3267753c5d'
                         	       }, 

      id='run-c94d8c7f-29bf-4873-b108-8bf4349dfc49', 

      tool_calls=[
                        {
                           'name': 'search_keyword', 
                           'args': {'query': '대구 동성로 떡볶이'}, 
                           'id': 'call_ipWysUL54d6KgDfUXjgRRsYP', 
                           'type': 'tool_call'
                        }
                    ], 

      tool_call_chunks=[
                                  {
                                        'name': 'search_keyword', 
                                        'args': '{"query":"대구 동성로 떡볶이"}', 
                                        'id': 'call_ipWysUL54d6KgDfUXjgRRsYP', 
                                        'index': 0, 
                                        'type': 'tool_call_chunk'
                                  }
                             ]
   )  // end AIMessageChunk
  ]  // end messages

}
"""

' \n{\n\n  \'actions\': \n  [\n    ToolAgentAction\n    (\n      tool=\'search_keyword\', \n\n      tool_input={\'query\': \'대구 동성로 떡볶이\'}, \n\n      log="\nInvoking: `search_keyword` with `{\'query\': \'대구 동성로 떡볶이\'}`\n\n\n", \n\n      message_log=\n      [\n        AIMessageChunk\n        (\n          content=\'\', \n          additional_kwargs=\n           {\n              \'tool_calls\': [\n                                 {\n                                    \'index\': 0, \n                                    \'id\': \'call_ipWysUL54d6KgDfUXjgRRsYP\', \n                                    \'function\': {\'arguments\': \'{"query":"대구 동성로 떡볶이"}\', \'name\': \'search_keyword\'}, \n                                    \'type\': \'function\'\n                                 }\n                            ]\n           }, \n\n           response_metadata={\'finish_reason\': \'tool_calls\', \'model_name\': \'gpt-4o-mini-2024-07-18\', \'system_fingerprint\': \'fp_3267753c5d\'}, \n \n   

In [ ]:
""" 
{
    'steps': 
    [AgentStep
     (
       action=ToolAgentAction(
                                          tool='search_keyword', 
                                          tool_input={'query': '대구 동성로 떡볶이'}, 
                                          log="\nInvoking: `search_keyword` with `{'query': '대구 동성로 떡볶이'}`\n\n\n", 
                                          message_log=[
                                                                AIMessageChunk(content='', 
                                                                                        additional_kwargs={
                                                                                                                     'tool_calls': [
                                                                                                                                      {
                                                                                                                                        'index': 0, 
                                                                                                                                        'id': 'call_iMcAsLCy9VJmh5l1e2cZJIso', 
                                                                                                                                        'function': {
                                                                                                                                                          'arguments': '{"query":"대구 동성로 떡볶이"}', 
                                                                                                                                                          'name': 'search_keyword'
                                                                                                                                                       }, 
                                                                                                                                        'type': 'function'
                                                                                                                                      }
                                                                                                                                   ]
                                                                                                                  }, 

                                                                                        response_metadata={
                                                                                                                       'finish_reason': 'tool_calls', 
                                                                                                                       'model_name': 'gpt-4o-mini-2024-07-18', 
                                                                                                                       'system_fingerprint': 'fp_3267753c5d'
                                                                                                                    }, 
                                                                                        id='run-be95c080-1eab-484c-98e4-5249c2ae031e', 
                                                                                        tool_calls=[
                                                                                                         {
                                                                                                           'name': 'search_keyword', 
                                                                                                           'args': {'query': '대구 동성로 떡볶이'}, 
                                                                                                           'id': 'call_iMcAsLCy9VJmh5l1e2cZJIso', 
                                                                                                           'type': 'tool_call'
                                                                                                         }
                                                                                                      ], 
                                                                                        tool_call_chunks=[
                                                                                                                   {
                                                                                                                      'name': 'search_keyword', 
                                                                                                                      'args': '{"query":"대구 동성로 떡볶이"}', 
                                                                                                                      'id': 'call_iMcAsLCy9VJmh5l1e2cZJIso', 
                                                                                                                      'index': 0, 
                                                                                                                      'type': 'tool_call_chunk'
                                                                                                                   }
                                                                                                                ]
                                                                )
                                                      ],
                                          tool_call_id='call_iMcAsLCy9VJmh5l1e2cZJIso'
             ), 


       observation=[ 
                             {
                                'url': 'https://news.google.com/rss/articles/CBMiaEFVX3lxTE8, 
                                'content': "'떡볶이 성지' 대구 북구, 역대급 '떡볶이 페스티벌' 막올랐다 - 대경일보"
                             }, 
                             
                             {
                                'url': 'https://news.google.com/rss/articles/CBMia0FVX3lxTE93', 
                                'content': '`탄핵 집회 참여자 주목` 대구 동성로 선결제 물결…커피·붕어빵·김밥 드세요 - 영남일보'
                             }, 

                             {
                                'url': 'https://news.google.com/rss/articles/CBMikAFBVV955', 
                                'content': '‘떡볶이.악동.활.선비’...어린이날 연휴 맞아 대구·경북 곳곳서 축제 - 조선일보'
                             }
                   ]
     )
    ], 

    'messages': 
    [
     FunctionMessage
     (
       content='[
                       {
                         "url": "https://news.google.com/rss/articles/CBMiaEFVX3lxTE83bE9", 
                         "content": "\'떡볶이 성지\' 대구 북구, 역대급 \'떡볶이 페스티벌\' 막올랐다 - 대경일보"
                       },
 
                       {
                         "url": "https://news.google.com/rss/articles/CBMia0FVX3lxTE93", 
                        "content": "`탄핵 집회 참여자 주목` 대구 동성로 선결제 물결…커피·붕어빵·김밥 드세요 - 영남일보"
                       }, 
   
                       {
                         "url": "https://news.google.com/rss/articles/CBMikAFBVV95", 
                        "content": "‘떡볶이.악동.활.선비’...어린이날 연휴 맞아 대구·경북 곳곳서 축제 - 조선일보"
                       }
                    ]', 
      additional_kwargs={}, 
      response_metadata={}, 
      name='search_keyword'
    )
   ]

}

// }
"""

' \n{\n    \'steps\': \n    [AgentStep\n     (\n       action=ToolAgentAction(\n                                          tool=\'search_keyword\', \n                                          tool_input={\'query\': \'대구 동성로 떡볶이\'}, \n                                          log="\nInvoking: `search_keyword` with `{\'query\': \'대구 동성로 떡볶이\'}`\n\n\n", \n                                          message_log=[\n                                                                AIMessageChunk(content=\'\', \n                                                                                        additional_kwargs={\n                                                                                                                     \'tool_calls\': [\n                                                                                                                                      {\n                                                                                                                

In [46]:
""" 
{
'output': 
             '대구 동성로 떡볶이에 대한 최근 뉴스는 다음과 같습니다:\n\n
             1. **[떡볶이 성지 대구 북구, 역대급 떡볶이 페스티벌 막올랐다 - 대경일보]
                (https://news.google.com/rss/articles/CBMiaEFVX3lxTE83bE9uX0hyTEJIV)**  \n   
                대구 북구에서 떡볶이 페스티벌이 열리고 있다는 소식입니다. 이 페스티벌은 떡볶이를 사랑하는 사람들에게 큰 인기를 끌고 있습니다.\n\n
             2. **[`탄핵 집회 참여자 주목` 대구 동성로 선결제 물결…커피·붕어빵·김밥 드세요 - 영남일보]                                     (https://news.google.com/rss/articles/CBMia0FVX3lxTE93M0J1SG9kMXJwUndseF9)**  \n   
                대구 동성로에서 선결제 문화가 확산되고 있으며, 다양한 먹거리를 즐길 수 있다는 내용입니다.\n\n
             3. **[‘떡볶이.악동.활.선비’...어린이날 연휴 맞아 대구·경북 곳곳서 축제 - 조선일보]
                (https://news.google.com/rss/articles/CBMikAFBVV95)**  \n   
                어린이날 연휴를 맞아 대구와 경북에서 다양한 축제가 열리고 있으며, 떡볶이와 관련된 행사도 포함되어 있습니다.\n\n
                이 뉴스들은 대구 동성로에서의 떡볶이와 관련된 다양한 행사와 문화에 대한 정보를 제공합니다.', 


'messages': 
                [
                  AIMessage(
                                   content='대구 동성로 떡볶이에 대한 최근 뉴스는 다음과 같습니다:\n\n
                                                1. **[떡볶이 성지 대구 북구, 역대급 떡볶이 페스티벌 막올랐다 - 대경일보]                                                                                                      (https://news.google.com/rss/articles/CBMiaEFVX3lxTE83bE9uX0hyTEJIV)**  \n   
                                                   대구 북구에서 떡볶이 페스티벌이 열리고 있다는 소식입니다. 이 페스티벌은 떡볶이를 사랑하는 사람들에게 큰 인기를 끌고 있습니다.\n
                                                2. **[`탄핵 집회 참여자 주목` 대구 동성로 선결제 물결…커피·붕어빵·김밥 드세요 - 영남일보]                                                                                                       (https://news.google.com/rss/articles/CBMia0FVX3lxTE93M0J1SG9kMXJwUndseF9)**  \n   
                                                   대구 동성로에서 선결제 문화가 확산되고 있으며, 다양한 먹거리를 즐길 수 있다는 내용입니다.\n\n
                                                3. **[‘떡볶이.악동.활.선비’...어린이날 연휴 맞아 대구·경북 곳곳서 축제 - 조선일보]
                                                   (https://news.google.com/rss/articles/CBMikAFBVV95)**  \n   
                                                   어린이날 연휴를 맞아 대구와 경북에서 다양한 축제가 열리고 있으며, 떡볶이와 관련된 행사도 포함되어 있습니다.\n\n
                                                   이 뉴스들은 대구 동성로에서의 떡볶이와 관련된 다양한 행사와 문화에 대한 정보를 제공합니다.', 
                                    additional_kwargs={}, 
                                    response_metadata={}
                  )
                ]

}

"""

" \n{\n'output': \n             '대구 동성로 떡볶이에 대한 최근 뉴스는 다음과 같습니다:\n\n\n             1. **[떡볶이 성지 대구 북구, 역대급 떡볶이 페스티벌 막올랐다 - 대경일보]\n                (https://news.google.com/rss/articles/CBMiaEFVX3lxTE83bE9uX0hyTEJIV)**  \n   \n                대구 북구에서 떡볶이 페스티벌이 열리고 있다는 소식입니다. 이 페스티벌은 떡볶이를 사랑하는 사람들에게 큰 인기를 끌고 있습니다.\n\n\n             2. **[`탄핵 집회 참여자 주목` 대구 동성로 선결제 물결…커피·붕어빵·김밥 드세요 - 영남일보]                                     (https://news.google.com/rss/articles/CBMia0FVX3lxTE93M0J1SG9kMXJwUndseF9)**  \n   \n                대구 동성로에서 선결제 문화가 확산되고 있으며, 다양한 먹거리를 즐길 수 있다는 내용입니다.\n\n\n             3. **[‘떡볶이.악동.활.선비’...어린이날 연휴 맞아 대구·경북 곳곳서 축제 - 조선일보]\n                (https://news.google.com/rss/articles/CBMikAFBVV95)**  \n   \n                어린이날 연휴를 맞아 대구와 경북에서 다양한 축제가 열리고 있으며, 떡볶이와 관련된 행사도 포함되어 있습니다.\n\n\n                이 뉴스들은 대구 동성로에서의 떡볶이와 관련된 다양한 행사와 문화에 대한 정보를 제공합니다.', \n\n\n'messages': \n                [\n                  AIMessage(\n                                   content='대구 동성로